In [1]:
import types
import pandas as pd

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from geopy.distance import distance

print('Libraries imported.')

Libraries imported.


In [2]:
# reads file containing adresses of numerelli
indirizzi=pd.read_excel('indirizzi.xlsx')

In [3]:
geolocator = Nominatim(user_agent="to_explorer")

In [4]:
# adds latitude and longitude info to dataframe
for i in range(len(indirizzi)):
    indirizzo = indirizzi.loc[i]['Indirizzo']
    
    location = geolocator.geocode(indirizzo)
    latitude = location.latitude
    longitude = location.longitude
    
    indirizzi.loc[i, 'lat'] = latitude
    indirizzi.loc[i, 'long'] = longitude

In [5]:
indirizzi

,Nome,Indirizzo,lat,long
0,Piero,"via dei Brusati, Roma",41.880854,12.415655
1,Federico,"via Lorenzo Rocci, Roma",41.863275,12.439772
2,FabrizioC,"via Jean Paul Sartre, Roma",41.837330,12.630502
3,FabrizioG,"Largo Sri Lanka, Roma",41.825432,12.471934
4,Gigi,"via Cena, Fiumicino",41.775807,12.226396
5,Sirio,"Via Caterina Troiani, Roma",41.813189,12.452846
6,Rocco,"via Valcamonica, Roma",41.932775,12.525885
7,Sergio,"via Bepi Romagnoni, Roma",41.775748,12.343886


In [6]:
# with k-means clusters indirizzi and gets the center for the cluster
lat_long=indirizzi[['lat','long']]
id_n=1
kmeans = KMeans(n_clusters=id_n, random_state=0).fit(lat_long)
id_label_uffici=kmeans.labels_
center = np.array(kmeans.cluster_centers_)

In [7]:
# plots numerelli position and the center of the cluster
address = 'Roma'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Roma using latitude and longitude values
map_Roma = folium.Map(location=[latitude, longitude], zoom_start=11)

label='Roma'
#maxdensita=Municipi['Densita'].max()

for lat, lng, name, in zip(indirizzi['lat'], 
                                   indirizzi['long'], 
                                   indirizzi['Nome']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Roma)  

for lat, long in center:
    label = 'Baricentro Numerelli {}'.format(center)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        location=[lat, long],
        #icon=folium.Icon(color=colors[center]),
        #icon=folium.Icon(color='red'),
        popup=label).add_to(map_Roma)
    
    
map_Roma